In [ ]:
import pandas as pd

# 1. CSV 파일 불러오기
df_4000TT = pd.read_csv('4000TT.csv')
df_input = pd.read_csv('input.csv')

# 2. 'machine_name'을 기준으로 병합
merged_df = pd.merge(df_4000TT, df_input, on='machine_name')

# 3. 'input' 컬럼만 추출하여 infoDF로 저장
infoDF = merged_df[['input']]

# 결과 확인
print(infoDF.head())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 예시: infoDF에 Address 열이 있다고 가정
# infoDF = pd.read_csv("your_file.csv")  # 필요시 파일 로딩

# 1. Address 열에서 행정동 추출 (보통 마지막 단어)
infoDF['행정동'] = infoDF['Address'].str.split().str[-1]

# 2. 행정동별 빈도 확인
print(infoDF['행정동'].value_counts())

# 3. 시각화 - countplot
plt.figure(figsize=(12, 6))
sns.countplot(data=infoDF, x='행정동', order=infoDF['행정동'].value_counts().index)
plt.xticks(rotation=45)
plt.xlabel('행정동')
plt.ylabel('빈도수')
plt.title('행정동 분포')
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Time_Driving (주행시간) 분포 시각화
plt.figure(figsize=(10, 5))
sns.histplot(data=infoDF, x='Time_Driving', kde=True)
plt.title('주행시간(Time_Driving) 분포')
plt.xlabel('Time_Driving')
plt.ylabel('빈도수')
plt.tight_layout()
plt.show()

# 2. Speed_Per_Hour (평균속도) 분포 시각화
plt.figure(figsize=(10, 5))
sns.histplot(data=infoDF, x='Speed_Per_Hour', kde=True)
plt.title('평균속도(Speed_Per_Hour) 분포')
plt.xlabel('Speed_Per_Hour')
plt.ylabel('빈도수')
plt.tight_layout()
plt.show()

In [ ]:
# 이상치 데이터 개수 확인
print("300 이상 속도 데이터 수:", (infoDF['Speed_Per_Hour'] >= 300).sum())

# 이상치 제거
infoDF = infoDF[infoDF['Speed_Per_Hour'] < 300]

# 결과 확인
print("처리 후 데이터 수:", len(infoDF))

import numpy as np

# 300 이상인 값은 300으로 바꾸기
infoDF['Speed_Per_Hour'] = np.where(infoDF['Speed_Per_Hour'] >= 300, 300, infoDF['Speed_Per_Hour'])

# 확인
print(infoDF['Speed_Per_Hour'].describe())

In [ ]:
# 조건에 맞는 데이터 필터링
infoDF = infoDF[
    (infoDF['Speed_Per_Hour'] >= 10) & (infoDF['Speed_Per_Hour'] <= 300) &
    (infoDF['Time_Driving'] >= 1) & (infoDF['Time_Driving'] <= 1000)
]

# 결과 확인
print("조건 적용 후 데이터 수:", len(infoDF))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 스케일러 생성
scaler = MinMaxScaler()

# 정규화 수행 및 새로운 컬럼 추가
infoDF[['Speed_Scaled', 'Time_Scaled']] = scaler.fit_transform(infoDF[['Speed_Per_Hour', 'Time_Driving']])

# 결과 확인
print(infoDF[['Speed_Per_Hour', 'Speed_Scaled', 'Time_Driving', 'Time_Scaled']].head())

In [ ]:
import pandas as pd

# 1. Address 컬럼에서 행정동 추출 (보통 마지막 단어)
infoDF['행정동'] = infoDF['Address'].str.split().str[-1]

# 2. 행정동 컬럼에 대해 원-핫 인코딩 수행
one_hot_df = pd.get_dummies(infoDF['행정동'], prefix='동')

# 3. infoDF에 인코딩된 컬럼 붙이기
infoDF = pd.concat([infoDF, one_hot_df], axis=1)

# 결과 확인
print(infoDF.head())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. X, y 정의
X = infoDF.drop(columns=['Time_Driving'])  # 특성
y = infoDF['Time_Driving']                # 타겟

# 2. 훈련/검증 데이터 분할 (80:20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. StandardScaler 적용
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. 결과를 DataFrame으로 정리 (원래 컬럼명 유지)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

# 5. 데이터 병합 (스케일된 X와 원래 y)
DataScaled_train = pd.concat([X_train_scaled_df, y_train], axis=1)
DataScaled_test = pd.concat([X_test_scaled_df, y_test], axis=1)

# 선택적으로 전체 스케일된 데이터를 합쳐 하나의 DataScaled로 만들 수도 있음
DataScaled = pd.concat([DataScaled_train, DataScaled_test])

# 결과 확인
print(DataScaled.head())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# 1. 피처(X)와 타겟(y) 분리
X = infoDF.drop(columns=['Time_Driving'])
y = infoDF['Time_Driving']

# 2. 학습/검증 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. 모델 정의 및 학습
dt_model = DecisionTreeRegressor(random_state=42)
rf_model = RandomForestRegressor(random_state=42)

dt_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

# 4. 예측
dt_pred = dt_model.predict(X_test)
rf_pred = rf_model.predict(X_test)

# 5. 성능 평가 (RMSE)
dt_rmse = np.sqrt(mean_squared_error(y_test, dt_pred))
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))

print(f"Decision Tree RMSE: {dt_rmse:.2f}")
print(f"Random Forest RMSE: {rf_rmse:.2f}")

# 6. 예측 결과 시각화
plt.figure(figsize=(12, 5))

# 의사결정나무
plt.subplot(1, 2, 1)
plt.scatter(y_test, dt_pred, alpha=0.5)
plt.xlabel("Actual Time_Driving")
plt.ylabel("Predicted (Decision Tree)")
plt.title("Decision Tree Prediction vs Actual")

# 랜덤포레스트
plt.subplot(1, 2, 2)
plt.scatter(y_test, rf_pred, alpha=0.5)
plt.xlabel("Actual Time_Driving")
plt.ylabel("Predicted (Random Forest)")
plt.title("Random Forest Prediction vs Actual")

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

# 1. MAE 계산
dt_mae = mean_absolute_error(y_test, dt_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)

# 2. 결과 출력
print(f"Decision Tree MAE: {dt_mae:.2f}")
print(f"Random Forest MAE: {rf_mae:.2f}")

# 3. 간단한 해석
if dt_mae < rf_mae:
    print("👉 의사결정나무 모델의 MAE가 더 낮으므로 더 정확한 예측을 수행했습니다.")
elif rf_mae < dt_mae:
    print("👉 랜덤포레스트 모델의 MAE가 더 낮으므로 더 정확한 예측을 수행했습니다.")
else:
    print("👉 두 모델의 MAE가 동일하거나 거의 비슷합니다.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# 1. 모델 구성
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),  # 입력층 + 은닉층1
    Dense(32, activation='relu'),                                    # 은닉층2
    Dense(1)                                                         # 출력층 (회귀 문제)
])

# 2. 모델 컴파일
model.compile(
    loss='mae',                        # 손실 함수: MAE
    optimizer='adam',                 # 최적화 도구: Adam
    metrics=['mae', 'mse']            # 평가지표: MAE, MSE
)

# 3. 학습 수행
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

# 4. 학습 결과 시각화
plt.figure(figsize=(12, 5))

# 손실(Loss)
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss (MAE)')
plt.plot(history.history['val_loss'], label='Val Loss (MAE)')
plt.title('Loss (MAE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.title('Mean Absolute Error (MAE)')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# MSE 시각화
plt.figure(figsize=(8, 5))
plt.plot(history.history['mse'], label='Train MSE')
plt.plot(history.history['val_mse'], label='Validation MSE')
plt.title('Model MSE Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
